In [1]:
import pandas as pd
from typing import Any
import re

## CLEANING TASKS <br>
1.	Standardize column names (snake_case) <br>
2.	Strip whitespace from all string fields <br>
3.	Normalize missing values (NaN, empty string, “-”, “NA”, “n/a”) <br>
4.	Convert Amount In USD to numeric <br>
5.	Convert Date to datetime <br>
6.	Clean city names (Bangalore/Bengaluru, Bombay/Mumbai, etc.) <br>
7.	Extract year from Date <br>
8.	Standardize Investment Type (Seed/Seed Funding → Seed) <br>
9.	Split multiple investors into lists <br>
10.	Clean Startup Name (remove suffixes like “ Pvt Ltd”, “ Limited”) <br>



In [2]:
df = pd.read_csv('/Users/olixstudios/Documents/workspace/Projects/indian-startups-transformations/data/starup.csv')
first_column_name = df.columns[0]
df = df.drop(columns=[first_column_name], errors="ignore")

In [3]:
def clean_column_names(df: pd.DataFrame) -> pd.DataFrame:
    def _snake_case(s: Any) -> str:
        if not isinstance(s, str):
            s = str(s)
        s  = s.strip()
        s = s.replace(" ", "_")
        s = re.sub(r"[^\w\s]", "", s)  
        return s.lower()
    df =  df.copy()
    df.columns = [_snake_case(col) for col in df.columns]
    return df

In [4]:
def normalize_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    MISSING_TOKENS = ["N/A", "n/a", "NA", "na", "-", "--", " ", ""]
    df = df.copy()
    df.replace(to_replace=MISSING_TOKENS, value=pd.NA, inplace=True)
    return df


In [5]:
# df["amount"].fillna(0)

In [6]:
# df["amount"] = df["amount"].str.replace(",", "")
# df["amount"] = pd.to_numeric(df["amount"], errors='coerce')
# df


In [7]:
def clean_amount_column(df: pd.DataFrame, col: str = "amount") -> pd.DataFrame:
    df = df.copy()
    df[col] = df[col].astype(str).fillna("")
    #df[col] = df[col].str.replace(",", "")
    df[col] = df[col].str.strip()
    #df[col] = df[col].str.replace(r"[\$,€£¥]", "", regex=True)
    df[col] = df[col].str.replace(r"[^0-9\.\-]", '', regex=True).replace('', pd.NA)
    df[col] = pd.to_numeric(df[col], errors='coerce')
    return df


In [8]:
def parse_dates(df: pd.DataFrame, col: str = "date") -> pd.DataFrame:
    df = df.copy()
    df[col] = pd.to_datetime(df[col], errors='coerce') 
    return df

In [9]:
def date_analysis(df: pd.DataFrame, date_col: str = "date") -> pd.DataFrame:
    df = df.copy()
    df["year"] = df[date_col].dt.year
    df["date_missing"] = df[date_col].isna()
    return df

In [10]:
print('orig cols:', df.columns.tolist())
df = clean_column_names(df)
print('clean cols:', df.columns.tolist())
df = normalize_missing_values(df)
print('null counts:', df.isna().sum().to_dict())
df = clean_amount_column(df, col='amount')   # or amount_in_usd
print('amount dtype:', df['amount'].dtype)
df = parse_dates(df, col='date')
print('date dtype:', df['date'].dtype)
print(df[['amount','date']].head().to_dict())

orig cols: ['Date', 'Startup', 'Industry', 'Location', 'Investor', 'Type', 'Amount']
clean cols: ['date', 'startup', 'industry', 'location', 'investor', 'type', 'amount']
null counts: {'date': 5, 'startup': 5, 'industry': 5, 'location': 5, 'investor': 35, 'type': 12, 'amount': 57}
amount dtype: float64
date dtype: datetime64[ns]
{'amount': {0: 460000000.0, 1: 300000000.0, 2: 343000000.0, 3: 83000000.0, 4: 7400000.0}, 'date': {0: Timestamp('2021-01-04 00:00:00'), 1: Timestamp('2021-05-04 00:00:00'), 2: NaT, 3: Timestamp('2021-07-04 00:00:00'), 4: NaT}}


In [ ]:
temp = df['type'].unique()


TypeError: '<' not supported between instances of 'float' and 'str'

AttributeError: 'Series' object has no attribute 'distinct'